In [0]:
# Pegar a variavel processdate do Data Factory

processdate = dbutils.widgets.get("processdate")
print(processdate)

In [0]:
mount_point = "/mnt/datalake"

# Check if the directory is already mounted
if len([mount for mount in dbutils.fs.mounts() if mount.mountPoint == mount_point]) == 0:
  dbutils.fs.mount(
      source="wasbs://datalake@ricaadls.blob.core.windows.net",
      mount_point=mount_point,
      extra_configs={
          "fs.azure.account.key.ricaadls.blob.core.windows.net": dbutils.secrets.get(scope="RicardoSecretScope", key="ricaAdlsSecret")
      }
  )

##Bibliotecas necessarias

In [0]:
from pyspark.sql.functions import lit

##Leitura e transformação

In [0]:
df = (spark
        .read
        .json('dbfs:/mnt/datalake/bronze/{}/*'.format(processdate))
)
# display(df)

In [0]:
data = df.withColumn('processdate', lit(str(processdate))).distinct()
# display(data)

###Silver

In [0]:
if "brewery_api" in [t.tableName.lower() for t in spark.sql("SHOW TABLES").select('tableName').collect()]:
    query = "DELETE FROM brewery_api WHERE processdate = '{0}'".format(processdate)
    dbutils.fs.rm('dbfs:/mnt/datalake/silver/{}'.format(processdate), True)
    _ = spark.sql(query)

(data
.write
.format('delta')
.partitionBy("processdate")
.mode('append')
.save('dbfs:/mnt/datalake/silver'))

spark.sql("CREATE TABLE IF NOT EXISTS brewery_api USING delta LOCATION 'dbfs:/mnt/datalake/silver'")

DataFrame[]

###Gold

In [0]:
data = spark.sql("""
    SELECT brewery_type, state, count(*) AS brewery_count
    FROM brewery_api
    GROUP BY brewery_type, state
""")

(data
.write
.format('delta')
.mode('overwrite')
.save('dbfs:/mnt/datalake/gold'))

In [0]:
%sql
CREATE OR REPLACE VIEW brewery_api_agg AS
SELECT brewery_type, state, count(*) AS brewery_count
FROM brewery_api
GROUP BY brewery_type, state;

In [0]:
%sql

SELECT * FROM brewery_api_agg

brewery_type,state,brewery_count
micro,Wisconsin,2
closed,California,1
large,Oregon,4
closed,Washington,1
micro,Virginia,1
micro,Massachusetts,1
micro,Iowa,1
micro,Oklahoma,1
micro,Michigan,1
micro,North Carolina,1
